# Analysis on IPL dataset - part 3

(Based on IPL matches and deliveries dataset)


<a>https://www.kaggle.com/nowke9/ipldata#matches.csv</a>


<a>https://www.kaggle.com/nowke9/ipldata#deliveries.csv<a>

In [0]:
import numpy as np
import pandas as pd
import io
from google.colab import files

In [0]:
match = pd.read_csv('/content/matches.csv')
delivery =pd.read_csv('/content/deliveries.csv')

In [0]:
match.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 756 entries, 0 to 755
Data columns (total 18 columns):
id                 756 non-null int64
season             756 non-null int64
city               749 non-null object
date               756 non-null object
team1              756 non-null object
team2              756 non-null object
toss_winner        756 non-null object
toss_decision      756 non-null object
result             756 non-null object
dl_applied         756 non-null int64
winner             752 non-null object
win_by_runs        756 non-null int64
win_by_wickets     756 non-null int64
player_of_match    752 non-null object
venue              756 non-null object
umpire1            754 non-null object
umpire2            754 non-null object
umpire3            119 non-null object
dtypes: int64(5), object(13)
memory usage: 106.4+ KB


In [0]:
delivery.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179078 entries, 0 to 179077
Data columns (total 21 columns):
match_id            179078 non-null int64
inning              179078 non-null int64
batting_team        179078 non-null object
bowling_team        179078 non-null object
over                179078 non-null int64
ball                179078 non-null int64
batsman             179078 non-null object
non_striker         179078 non-null object
bowler              179078 non-null object
is_super_over       179078 non-null int64
wide_runs           179078 non-null int64
bye_runs            179078 non-null int64
legbye_runs         179078 non-null int64
noball_runs         179078 non-null int64
penalty_runs        179078 non-null int64
batsman_runs        179078 non-null int64
extra_runs          179078 non-null int64
total_runs          179078 non-null int64
player_dismissed    8834 non-null object
dismissal_kind      8834 non-null object
fielder             6448 non-null object
dtype

## Q1. Find out a City which has hosted most no. of IPL openers i.e first match of a season.

In [0]:
cities=match.drop_duplicates(subset='season',keep='first')['city'].value_counts().sort_values(ascending=False).reset_index()
max_hosts=cities['city'].max()
cities[cities['city']==max_hosts].rename(columns={'index':'City','city':'Times_Hosted'})

,City,Times_Hosted
0,Chennai,3
1,Mumbai,3


## Q2. Find out top-3 batsman who has scored most runs against Malinga.

In [0]:
def check_name(series,name_like):
    series.unique()
    for st in series.unique():
        if name_like.lower() in st.lower(): 
            print(st)
    
# Checking for full name of Malinga    
check_name(delivery['bowler'],'mal')

SL Malinga
Shoaib Malik
VS Malik
I Malhotra


In [0]:
delivery[delivery['bowler']=='SL Malinga'].groupby('batsman').sum()['batsman_runs'].sort_values(ascending=False).head(3).reset_index()

,batsman,batsman_runs
0,SR Watson,127
1,AB de Villiers,125
2,MS Dhoni,89


## Q3. Make a function to find top bowler(in terms of most wickets) against any team. Take input as team.

In [0]:
delivery['dismissal_kind'].unique()

array([nan, 'caught', 'bowled', 'run out', 'lbw', 'caught and bowled',
       'stumped', 'retired hurt', 'hit wicket', 'obstructing the field'],
      dtype=object)

In [0]:
def top_bowler(team):
    
    name={
            "MI" : "Mumbai Indians",
            "CSK" : "Chennai Super Kings",            
            "KKR" : "Kolkata Knight Riders",          
            "RCB" : "Royal Challengers Bangalore",
            "KXIP" : "Kings XI Punjab",               
            "RR" : "Rajasthan Royals",               
            "DD" : "Delhi Daredevils",               
            "SH" : "Sunrisers Hyderabad",            
            "DC" : "Deccan Chargers",                
            "GL" : "Gujarat Lions",                  
            "PW" : "Pune Warriors",                  
            "RPS" : "Rising Pune Supergiant",         
            "KTK" : "Kochi Tuskers Kerala",            
            "RPSS" : "Rising Pune Supergiants", 
            "DC" : 'Delhi Capitals'
            }
    team=name.get(team.upper(),team)
    
    return delivery[(delivery['dismissal_kind'].isin(['caught', 'bowled', 'lbw', 'caught and bowled','stumped'])) & (delivery['batting_team']==team)].groupby('bowler').size().sort_values(ascending=False).head(1).reset_index().rename(columns={'bowler':'Bowler',0:'Wickets'})

In [0]:
top_bowler('rcb')

,Bowler,Wickets
0,A Nehra,23


In [0]:
top_bowler('csk')

,Bowler,Wickets
0,SL Malinga,31


## Q4. Write a function which will take team name as input and return it's record of no. occurance in finals and no. of times won.  [Difficult]

ex.-- if team is CSK-- 
CSK in finals-

Played- 8
Won- 3

If any team which has not played in finals like Gujrat Lions

Gujrat Lions in finals-

Played-0


In [0]:
def finals_and_wins(team):
    
    name={
            "MI" : "Mumbai Indians",
            "CSK" : "Chennai Super Kings",            
            "KKR" : "Kolkata Knight Riders",          
            "RCB" : "Royal Challengers Bangalore",
            "KXIP" : "Kings XI Punjab",               
            "RR" : "Rajasthan Royals",               
            "DD" : "Delhi Daredevils",               
            "SH" : "Sunrisers Hyderabad",            
            "DC" : "Deccan Chargers",                
            "GL" : "Gujarat Lions",                  
            "PW" : "Pune Warriors",                  
            "RPS" : "Rising Pune Supergiant",         
            "KTK" : "Kochi Tuskers Kerala",            
            "RPSS" : "Rising Pune Supergiants", 
            "DC" : 'Delhi Capitals'
            }
    team=name.get(team.upper(),team)
    
    
    
    winners=match.drop_duplicates(subset=['season'],keep='last')
    if (winners[(winners['team1']==team) | (winners['team2']==team)].shape[0])==0:
        print(team,'played 0 matches in IPL Finals')
    else:
        finals=winners[(winners['team1']==team) | (winners['team2']==team)].shape[0]
        wins=winners[winners['winner']==team].shape[0]
        print(team,'played',finals,'matches in finals and won',wins,'times')

In [0]:
finals_and_wins('Kkr')

Kolkata Knight Riders played 2 matches in finals and won 2 times


In [0]:
finals_and_wins('Gujarat Lions')

Gujarat Lions played 0 matches in IPL Finals


In [0]:
finals_and_wins('csk')

Chennai Super Kings played 8 matches in finals and won 3 times


## Q5. Write a function which will take a input as batsman and in output it gives total runs scored against each team. [Medium]




In [0]:
def total_runs(batsman):
    return delivery[delivery['batsman']==batsman].groupby('bowling_team').sum()['total_runs'].sort_values(ascending=False).reset_index().rename(columns={'bowling_team':'Opponent Team','total_runs':'Total_Runs Scored'})

In [0]:
total_runs('V Kohli')

,Opponent Team,Total_Runs Scored
0,Delhi Daredevils,789
1,Chennai Super Kings,777
2,Kolkata Knight Riders,685
3,Kings XI Punjab,661
4,Mumbai Indians,652
5,Sunrisers Hyderabad,528
6,Rajasthan Royals,382
7,Deccan Chargers,323
8,Gujarat Lions,287
9,Rising Pune Supergiants,201
